## 1. 환경 구성

### 1) 라이브러리 설치

In [2]:
!pip install -q langchain langchain-openai tiktoken python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2) OpenAI 인증키 설정

https://openai.com/

In [8]:
import os

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
# os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'

## 2. LLM Chain

### 1) Prompt + LLM
- 가장 기본적이고 일반적인 사용 사례
- 프롬프트 템플릿과 모델을 연결

In [ ]:
from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

# chain 실행
resp = llm.invoke('지구의 자전 주기는?')  # AIMessage object

In [ ]:
resp.content

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# promt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy, Answer the question, <Question>: {input}")
llm = ChatOpenAI(model='gpt-3.5-turbo-0125')
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

### 2) Multiple Chains

In [ ]:
prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English")
prompt2 = ChatPromptTemplate.from_template("explain {english_word} using oxford dictionary to me in Korean.")

llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({"korean_word": "미래"})

In [ ]:
chain2 = (
    {"english_word": chain1}
    | prompt2
    | llm
    | StrOutputParser()    
)

chain2.invoke({"korean_word": "미래"})

## 3. Prompt
- 프롬프트: 사용자가 언어 모델에 입력하는 질문이나 요청
- 모델이 제공할 응답의 방향과 내용을 결정하는 데 중요한 역할
- 프롬프트 작성 원칙
    1. 명확성과 구체성
        - 질문은 명확하고 구체적이어야 합니다. 모호한 질문은 LLM 모델의 혼란을 초래할 수 있기 때문입니다.
        - 예시: "다음 주 주식 시장에 영향을 줄 수 있는 예정된 이벤트들은 무엇일까요?"는 "주식 시장에 대해 알려주세요."보다 더 구체적이고 명확한 질문입니다.
    2. 배경 정보를 포함
        - 모델이 문맥을 이해할 수 있도록 필요한 배경 정보를 제공하는 것이 좋습니다. 이는 환각 현상(hallucination)이 발생할 위험을 낮추고, 관련성 높은 응답을 생성하는 데 도움을 줍니다.
        - 예시: "2020년 미국 대선의 결과를 바탕으로 현재 정치 상황에 대한 분석을 해주세요."
    3. 간결함
        - 핵심 정보에 초점을 맞추고, 불필요한 정보는 배제합니다. 프롬프트가 길어지면 모델이 덜 중요한 부분에 집중하거나 상당한 영향을 받는 문제가 발생할 수 있습니다.
        - 예시: "2021년에 발표된 삼성전자의 ESG 보고서를 요약해주세요."
    4. 열린 질문 사용
        - 열린 질문을 통해 모델이 자세하고 풍부한 답변을 제공하도록 유도합니다. 단순한 '예' 또는 '아니오'로 대답할 수 있는 질문보다는 더 많은 정보를 제공하는 질문이 좋습니다.
        - 예시: "신재생에너지에 대한 최신 연구 동향은 무엇인가요?"
    5. 명확한 목표 설정
        - 얻고자 하는 정보나 결과의 유형을 정확하게 정의합니다. 이는 모델이 명확한 지침에 따라 응답을 생성하도록 돕습니다.
        - 예시: "AI 윤리에 대한 문제점과 해결 방안을 요약하여 설명해주세요."
    6. 언어와 문체
        - 대화의 맥락에 적합한 언어와 문체를 선택합니다. 이는 모델이 상황에 맞는 표현을 선택하는데 도움이 됩니다.
        - 예시: 공식적인 보고서를 요청하는 경우, "XX 보고서에 대한 전문적인 요약을 부탁드립니다."와 같이 정중한 문체를 사용합니다.

### 1) PromptTemplate
- PromptTemplate + LLMs (단일 문장 입력 -> 단일 문장 출력)
- **문자열 프롬프트**를 위한 템플릿을 생성. Python의 문자열 포맷팅 구문을 사용.
- 내용: 지시사항, 몇 가지 예시, 특정 맥락 및 질문

### 2) ChatPromptTemplate
- chatPromptTemplates + ChatModels (여러 메시지 입력 -> 단일 메시지 출력)
- 채팅 메시지를 원소로 갖는 리스트 형태
- 구성: 각 채팅 메시지는 역할(role)과 내용(content)이 짝을 이루는 형태
    - 예시: OpenAI는 AI Assistant, Human, System 등의 역할(role)로 구성

1. Message 유형
- SystemMessage: 시스템의 기능을 설명합니다.
- HumanMessage: 사용자의 질문을 나타냅니다.
- AIMessage: AI 모델의 응답을 제공합니다.
- FunctionMessage: 특정 함수 호출의 결과를 나타냅니다.
- ToolMessage: 도구 호출 결과를 나타냅니다.

In [ ]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    {"system": "이 시스템은 천문학 질문에 답변할 수 있습니다."},
    {"user": "{user_input}"},
])

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇일까요?")
messages

In [ ]:
# 3-MessagePromptTemplate 활용
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."),
    HumanMessagePromptTemplate.from_template("{user_input}"),
])

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇일까요?")
messages

In [ ]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇일까요?"})

## 4. Model Parameter

### 1) 파라미터 설정
- Temperature: 값이 작으면 예측 가능하고 일관된 출력을 생성하는 반면, 값이 크면 다양하고 예측하기 어려운 출력을 생성합니다.
- Max Tokens(최대 토큰 수): 생성할 텍스트의 길이를 제한합니다.
- Top P(Probability): 특정 확률 분포 내에서 상위 P% 토큰만을 고려하는 방식입니다.
- Frequency Penalty: 값이 클수록 이미 등장한 단어나 구절이 다시 등장할 확률을 감소시킨다.
- Presence Penalty: 값이 클수록 아직 텍스트에 등장하지 않은 새로운 단어의 사용이 장려됩니다. 
- Stop Sequences: 출력을 특정 포인트에서 종료하고자 할 때 사용합니다.

In [ ]:
# 모델 파라미터 설정
params = {
    "temperature": 0.7,
    "max_tokens": 100,
}

kwargs = {
    "frequency_penalty": 0.5,
    "presence_penalty": 0.5,
    "stop": ["\n"],
}

# 모델 인스턴스를 생성할 때 설정
model = ChatOpenAI(model='gpt-3.5-turbo-0125', **params, model_kwargs=kwargs)

# 모델 호출
q = "태양계에서 가장 큰 행성은 무엇인가요?"
resp = model.invoke(q)

resp


In [ ]:
# 모델 인스턴스를 호출할 때 전달
params = {
    "temperature": 0.7,
    "max_tokens": 100,
}

resp = model.invoke(input=q, **params)

resp

모델에 추가적인 파라미터 전달
- bind 메서드를 사용하여 체인에 새로운 파라미터를 추가하여 연결 가능
- 다양한 상황에 맞게 모델의 동작을 제어

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."),
    HumanMessagePromptTemplate.from_template("{user_input}"),
])

model = ChatOpenAI(model='gpt-3.5-turbo-0125', max_tokens=100)

messages = prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇일까요?")

before_answer = model.invoke(messages)

print(before_answer)

# 모델 호출 시, 추가적인 인수를 전달하기 위해 bind 메서드 사용
chain = prompt | model.bind(max_tokens=10)

after_answer = chain.invoke(messages)

print(after_answer)